In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

In [ ]:


class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

# 假设 processed_data 和 labels 是预处理后的数据和标签
dataset = CustomDataset(processed_data, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
